# Exercises from Think Stats, 2nd Edition

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT


In [20]:
from __future__ import print_function, division

%matplotlib inline

import numpy as np
import pandas as pd

import random

import thinkstats2
import thinkplot

import statsmodels.formula.api as smf

## Exercises

**Exercise:** Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [62]:
import first
live, firsts, others = first.MakeFrames()
live_30 = live[live.prglngth>30]

In [63]:
import patsy

def GoMining(df):
    """Searches for variables that predict pregnancy length.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = 'prglngth ~ ' + name
            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError):
            continue

        variables.append((results.rsquared, name))

    return variables

In [64]:
variables = GoMining(live_30)

The following functions report the variables with the highest values of $R^2$.

In [65]:
import re

def ReadVariables():
    """Reads Stata dictionary files for NSFG data.

    returns: DataFrame that maps variables names to descriptions
    """
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars
    

def MiningReport(variables, n=45):
    """Prints variables with the highest R^2.

    t: list of (R^2, variable name) pairs
    n: number of pairs to print
    """
    all_vars = ReadVariables()

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except (KeyError, IndexError):
            print(name, r2)

In [66]:
# Listing out the explanatory variables that could be available before pregnancy for all. 

# 1) birthord    BIRTH ORDER
# 2) poverty     POVERTY LEVEL INCOME
# 3) pregordr    PREGNANCY ORDER (NUMBER)
# 4) educat      EDUCATION (COMPLETED YEARS OF SCHOOLING)
# 5) hieduc      HIGHEST COMPLETED YEAR OF SCHOOL OR DEGREE
# 6) race        RACE
# 7) nbrnaliv    BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
MiningReport(variables)

prglngth 1.0 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.8062434116139234 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
totalwgt_lb 0.12445743148120247
birthwgt_lb 0.11977307804917214 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.10372542204583346 LOW BIRTHWEIGHT - BABY 1
mosgest 0.09562431989592657 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
prglngth_i 0.0220537757964685 PRGLNGTH IMPUTATION FLAG
nbrnaliv 0.004577565785532922 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
anynurse 0.0024520248837112124 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bfeedwks 0.0023691839446674523 DURATION OF BREASTFEEDING IN WEEKS
pregend1 0.0022493894337981546 BC-1 HOW PREGNANCY ENDED - 1ST MENTION
cmlastlb 0.0020431424422022726 CM FOR R'S MOST RECENT LIVE BIRTH
fmarcon5_i 0.0019681593242578677 FMARCON5 IMPUTATION FLAG
evuseint 0.0018917527758618435 EG-1 USE ANY METHOD IN PREGNANCY INTERVAL?
gestasun_m 0.0016571319550165997 BC-5 GESTATIO

The following are the only variables I found that have a statistically significant effect on pregnancy length. Also these variables are known before the birth and also for all.

In [67]:
# Creating a new dataframe using the above explanatory variables along with the dependant variable
# 'prglngth','birthord','poverty','pregordr','educat','hieduc','race'
live2 = live_30[['prglngth','birthord','poverty','pregordr','educat','race','nbrnaliv']]
live2.describe()

,prglngth,birthord,poverty,pregordr,educat,race,nbrnaliv
count,8884.000000,8884.000000,8884.000000,8884.000000,8884.000000,8884.000000,8880.000000
mean,38.878095,1.826542,208.030842,2.271387,12.542211,1.826880,1.019144
std,1.898084,1.040100,145.340008,1.433924,2.555545,0.566995,0.176546
min,31.000000,1.000000,7.000000,1.000000,9.000000,1.000000,1.000000
25%,39.000000,1.000000,84.000000,1.000000,11.000000,1.000000,1.000000
50%,39.000000,2.000000,164.000000,2.000000,12.000000,2.000000,1.000000
75%,40.000000,2.000000,313.000000,3.000000,14.000000,2.000000,1.000000
max,50.000000,10.000000,500.000000,17.000000,19.000000,3.000000,5.000000


In [68]:
# Get the number of rows and columns in the new dataframe
# It shows we have 8884 rows and 7 columns now
live2.shape

(8884, 7)

In [69]:
# Get the value counts from the categorical columns

live2['poverty'].value_counts()

469    477
156    415
500    340
396    311
64     268
      ... 
57       3
7        3
126      2
498      1
415      1
Name: poverty, Length: 136, dtype: int64

In [70]:
# Using the above mentioned attributes to generate a model
# model = smf.ols('prglngth ~ birthord==1 + race==2 + nbrnaliv>1', data=live)

model = smf.ols('prglngth ~ birthord==1 + poverty > 164 + pregordr==1 + educat > 12 + race ==2 + nbrnaliv>1', data = live2)

results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     18.06
Date:                Sun, 19 Jul 2020   Prob (F-statistic):           6.01e-21
Time:                        16:27:17   Log-Likelihood:                -18245.
No. Observations:                8884   AIC:                         3.650e+04
Df Residuals:                    8877   BIC:                         3.655e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.7357      0.042    922.693      0.000      38.653      38.818
birthord == 1[T.True]     0.1785      0.066      2.703      0.007       0.049       0.308
poverty > 164[T.True]     0.0399      0.043      0.929      0.353      -0.044       0.124
pregordr == 1[T.True]    -0.1099      0.068     -1.614      0.107      -0.243       0.024
educat > 12[T.True]       0.0411      0.043      0.959      0.337      -0.043       0.125
race == 2[T.True]         0.1269      0.043      2.974      0.003       0.043       0.211
nbrnaliv > 1[T.True]     -1.4911      0.165     -9.064      0.000      -1.814      -1.169
==============================================================================
Omnibus:                     1603.180   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6201.864
Skew:                          -0.861   Prob(JB):                         0.00
Kurtosis:                       6.713   Cond. No.                         12.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Exercise:** If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [71]:
# Solution goes here
# loading the Pregnancy data and combining wth the nsfg data
# In order to predict the number of babies which is available in the nsfg data
import nsfg

live, firsts, others = first.MakeFrames()

resp = nsfg.ReadFemResp()
resp.index = resp.caseid
combined_df = live.join(resp, on='caseid', rsuffix='_r')

combined_df.columns

Index(['caseid', 'pregordr', 'howpreg_n', 'howpreg_p', 'moscurrp', 'nowprgdk',
       'pregend1', 'pregend2', 'nbrnaliv', 'multbrth',
       ...
       'pubassis_i_r', 'basewgt_r', 'adj_mod_basewgt_r', 'finalwgt_r',
       'secu_r', 'sest_r', 'cmintvw_r', 'cmlstyr', 'screentime', 'intvlngth'],
      dtype='object', length=3331)

In [72]:
# Solution goes here
# Creating a subset of the dataset using only the dependant variable and the explanatory variables from the combined dataframe

df1=combined_df[['age_r','race','totincr','educat','numbabes']]

Now we can predict the number of children for a woman who is 35 years old, black, and a college
graduate whose annual household income exceeds $75,000

In [73]:
# Solution goes here
# formula1 is linear
formula1='numbabes ~ age_r + C(race) + totincr + educat'
# Formula2 is making it non linear by taking a square of the age
combined_df['age2'] = combined_df.age_r**2
formula2='numbabes ~ age_r + age2 + C(race) + totincr + educat'

In [74]:
model = smf.poisson(formula1, data=combined_df)
results1 = model.fit()
results1.summary()

Optimization terminated successfully.
         Current function value: 1.688168
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 9148
Model:                        Poisson   Df Residuals:                     9142
Method:                           MLE   Df Model:                            5
Date:                Sun, 19 Jul 2020   Pseudo R-squ.:                 0.03060
Time:                        16:28:03   Log-Likelihood:                -15443.
converged:                       True   LL-Null:                       -15931.
Covariance Type:            nonrobust   LLR p-value:                1.514e-208
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.0797      0.044     24.278      0.000       0.993       1.167
C(race)[T.2]    -0.1418      0.014     -9.806      0.000      -0.170      -0.113
C(race)[T.3]    -0.0912      0.024     -3.765      0.000      -0.139      -0.044
age_r            0.0206      0.001     20.697      0.000       0.019       0.023
totincr         -0.0178      0.002     -9.531      0.000      -0.022      -0.014
educat          -0.0436      0.003    -15.063      0.000      -0.049      -0.038
================================================================================
"""

In [75]:
# Prediction based on Model 1
columns = ['age_r', 'race', 'totincr', 'educat']
new = pd.DataFrame([[35, 1, 14, 16]], columns=columns)
results1.predict(new)

0    2.352303
dtype: float64

In [76]:
model = smf.poisson(formula2, data=combined_df)
results2 = model.fit()
results2.summary()

Optimization terminated successfully.
         Current function value: 1.678215
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 9148
Model:                        Poisson   Df Residuals:                     9141
Method:                           MLE   Df Model:                            6
Date:                Sun, 19 Jul 2020   Pseudo R-squ.:                 0.03632
Time:                        16:28:06   Log-Likelihood:                -15352.
converged:                       True   LL-Null:                       -15931.
Covariance Type:            nonrobust   LLR p-value:                9.041e-247
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.0197      0.166     -6.132      0.000      -1.346      -0.694
C(race)[T.2]    -0.1422      0.014     -9.827      0.000      -0.171      -0.114
C(race)[T.3]    -0.0980      0.024     -4.047      0.000      -0.146      -0.051
age_r            0.1544      0.010     15.157      0.000       0.134       0.174
age2            -0.0020      0.000    -13.230      0.000      -0.002      -0.002
totincr         -0.0186      0.002     -9.904      0.000      -0.022      -0.015
educat          -0.0464      0.003    -15.994      0.000      -0.052      -0.041
================================================================================
"""

In [77]:
# Prediction based on Model 2
columns = ['age_r', 'age2','race', 'totincr', 'educat']
new = pd.DataFrame([[35,35**2, 1, 14, 16]], columns=columns)
results2.predict(new)

0    2.507703
dtype: float64

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?
Ans: 2 or 3

**Exercise:** If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called `mnlogit`. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called `rmarital`.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [78]:
# Solution goes here
# building a new dataframe with the relevant explanatory variables that are significant

df_logit=combined_df[['age_r','age2','race','totincr','educat','rmarital']]
df_logit.shape

(9148, 6)

In [79]:
df_logit.describe

<bound method NDFrame.describe of        age_r  age2  race  totincr  educat  rmarital
0         44  1936     2       14      16         1
1         44  1936     2       14      16         1
2         20   400     1        4      11         6
3         20   400     1        4      11         6
4         20   400     1        4      11         6
...      ...   ...   ...      ...     ...       ...
13581     35  1225     3        8      17         1
13584     31   961     2        8      12         1
13588     37  1369     2       10      13         1
13591     37  1369     2       10      13         1
13592     37  1369     2       10      13         1

[9148 rows x 6 columns]>

In [80]:
# building the model
formula = 'rmarital ~ age_r + age2 + C(race) + totincr + educat'
model = smf.mnlogit(formula, data = df_logit)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.092083
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 9148
Model:                        MNLogit   Df Residuals:                     9113
Method:                           MLE   Df Model:                           30
Date:                Sun, 19 Jul 2020   Pseudo R-squ.:                  0.1661
Time:                        16:28:15   Log-Likelihood:                -9990.4
converged:                       True   LL-Null:                       -11981.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        8.9153      0.792     11.251      0.000       7.362      10.468
C(race)[T.2]    -0.9260      0.087    -10.705      0.000      -1.096      -0.756
C(race)[T.3]    -0.6335      0.133     -4.747      0.000      -0.895      -0.372
age_r           -0.3567      0.050     -7.132      0.000      -0.455      -0.259
age2             0.0047      0.001      6.054      0.000       0.003       0.006
totincr         -0.1301      0.011    -11.475      0.000      -0.152      -0.108
educat          -0.1940      0.018    -10.534      0.000      -0.230      -0.158
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.9927      2.970      1.007      0.314      -2.829       8.815
C(race)[T.2]    -0.3963      0.235     -1.685      0.092      -0.857       0.065
C(race)[T.3]     0.0650      0.336      0.194      0.846      -0.593       0.723
age_r           -0.3141      0.174     -1.806      0.071      -0.655       0.027
age2             0.0064      0.003      2.532      0.011       0.001       0.011
totincr         -0.3217      0.032    -10.135      0.000      -0.384      -0.259
educat          -0.1093      0.048     -2.266      0.023      -0.204      -0.015
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -3.6475      1.193     -3.059      0.002      -5.985      -1.310
C(race)[T.2]    -0.3303      0.091     -3.641      0.000      -0.508      -0.153
C(race)[T.3]    -0.8227      0.170     -4.853      0.000      -1.155      -0.490
age_r            0.1238      0.070      1.763      0.078      -0.014       0.261
age2            -0.0008      0.001     -0.814      0.416      -0.003       0.001
totincr         -0.2288      0.011    -20.058      0.000      -0.251      -0.206
educat           0.0661      0.016      4.015      0.000       0.034       0.098
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.3978      1.269     -1.889      0.059      -4.886       0.090
C(race)[T.2]    -1.0493      0.101    -10.366      0.000      -1.248      -0.851
C(race)[T.3]    -0.6065      0.154     -3.937      0.000      -0.908      -0.305
age_r            0.2084      0.077      2.699      0.007       0.057       0.360
age2            -0.0030      0.001     -2.619      0.009      -0.005      -0.001
totincr         -0.2900      0.014    -20.314      0.000      -0.318      -0.262
educat          -0.0176      0.021     -0.835      0.404      -0.059       0.024
-

Make a prediction for a woman who is 25 years old, white, and a high
school graduate whose annual household income is about $45,000.

In [81]:
# Solution goes here
columns = ['age_r','age2','race','totincr','educat']
d = pd.DataFrame([[25, 25**2,2,11,12]], columns=columns)
results.predict(d)

,0,1,2,3,4,5
0,0.745689,0.128291,0.001624,0.032801,0.021887,0.069708


As per the above output 
This woman has a 75% chance of being currently married,
a 13% chance of being "not married but living with opposite sex partner" 